In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
#!/usr/bin/env python3
"""
Transfer Learning
"""

import tensorflow.keras as K
import tensorflow as tf
import numpy as np

def preprocess_data(X, Y):
    """
    * X is a numpy.ndarray of shape (m, 32, 32, 3) containing
        the CIFAR 10 data, where m is the number of data points
    * Y is a numpy.ndarray of shape (m,) containing the CIFAR 10
        labels for X
        Returns: X_p, Y_p
    * X_p is a numpy.ndarray containing the preprocessed X
    * Y_p is a numpy.ndarray containing the preprocessed Y
    """
    
    entrada = K.Input(shape=(32, 32, 3))
    resize = K.layers.Lambda(lambda image: tf.image.resize(image, (150, 150)))(entrada)
    dense169 = K.applications.DenseNet169(include_top=False, weights="imagenet", input_tensor=resize)
    out = dense169(resize)
    pre_model = K.models.Model(inputs=entrada, outputs=out)

    X_p = K.applications.densenet.preprocess_input(X)
    features = pre_model.predict(X_p)
    Y_p = K.utils.to_categorical(y=Y, num_classes=10)
    return (features, Y_p)


In [3]:
  # getting info
  (x_train, y_train), (x_test, y_test) = K.datasets.cifar10.load_data()
  # preprocessing
  x_p, y_p = preprocess_data(x_train, y_train)
  X_p, Y_p = preprocess_data(x_test, y_test)


170500096/170498071 [==============================] - 2s 0us/step
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
51879936/51877672 [==============================] - 0s 0us/step


In [4]:
if __name__ == "__main__":

    lay_init = K.initializers.he_normal()

    new_input = K.Input(shape=x_p.shape[1:])
    vector = K.layers.Flatten()(new_input)
    
    drop1 = K.layers.Dropout(0.3)(vector)
    norm_lay1 = K.layers.BatchNormalization()(drop1)
    FC1 = K.layers.Dense(units=510, activation='relu', kernel_initializer=lay_init)(norm_lay1)
    norm_lay2 = K.layers.BatchNormalization()(FC1)
    out = K.layers.Dense(units=10, activation='softmax', kernel_initializer=lay_init)(norm_lay2)
 
    model = K.models.Model(inputs=new_input, outputs=out)
    
    learn_dec = K.callbacks.ReduceLROnPlateau(monitor='val_acc', factor=0.1, patience=2)
   
    early = K.callbacks.EarlyStopping(patience=5)
    save = K.callbacks.ModelCheckpoint(filepath='cifar10.h5', save_best_only=True, monitor='val_acc', mode='max')

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(x=x_p, y=y_p, batch_size=32, epochs=15, verbose=1, validation_data=(X_p, Y_p), callbacks=[save, early, learn_dec])

Train on 50000 samples, validate on 10000 samples
Epoch 1/15
50000/50000 [==============================] - 34s 679us/sample - loss: 0.3951 - acc: 0.8706 - val_loss: 0.2800 - val_acc: 0.9026
Epoch 2/15
50000/50000 [==============================] - 24s 486us/sample - loss: 0.2177 - acc: 0.9246 - val_loss: 0.2662 - val_acc: 0.9080
Epoch 3/15
50000/50000 [==============================] - 25s 498us/sample - loss: 0.1506 - acc: 0.9466 - val_loss: 0.2705 - val_acc: 0.9142
Epoch 4/15
50000/50000 [==============================] - 24s 475us/sample - loss: 0.1130 - acc: 0.9599 - val_loss: 0.2964 - val_acc: 0.9104
Epoch 5/15
50000/50000 [==============================] - 26s 512us/sample - loss: 0.0813 - acc: 0.9720 - val_loss: 0.3126 - val_acc: 0.9115
Epoch 6/15
50000/50000 [==============================] - 24s 484us/sample - loss: 0.0420 - acc: 0.9860 - val_loss: 0.2739 - val_acc: 0.9233
Epoch 7/15
50000/50000 [==============================] - 24s 482us/sample - loss: 0.0276 - acc: 0.9916 

In [5]:

modelL = K.models.load_model('cifar10.h5')
modelL.evaluate(X_p, Y_p, batch_size=128, verbose=1)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
10000/10000 [==============================] - 2s 213us/sample - loss: 0.2675 - acc: 0.9256


[0.2674740501880646, 0.9256]